### Library

In [1]:
import requests
import json

import pandas as pd
import time

### Fungsi untuk mengambil data dari API

In [2]:
link ='https://api.opendota.com/api/publicMatches'
# link = 'https://api.opendota.com/api/proMatches'

def fetch_newdata():
    r = requests.get(link)
    data = json.loads(r.text) # load json data
    
    new_df = pd.DataFrame(data)                    # save json data to dafaframe
    new_df = new_df[(new_df['avg_mmr'] > 1999 ) &  # filter dataframe 
                    (new_df['lobby_type'] == 7) &
                    (new_df['game_mode'] == 22)] 
    
    return new_df

### Start program selama running time

In [3]:
running_time = 0.1                            # in hour
start_time = time.time()

df = pd.DataFrame()                         # make new data frame
# df = pd.read_csv('new_raw_data.csv')      # continue
rows_count_before = df.shape[0]
new_df = pd.DataFrame()  
call_counter = 0
error_counter = 0
    
while True:
    call_counter += 1
    
    new_df = fetch_newdata()
    df = pd.concat([df, new_df]).drop_duplicates('match_id').reset_index(drop=True)      # add new data
    df.to_csv('new_data.csv', sep=',', index=False)                                      # save new data
   
    time.sleep(60)                                 # fetching data every 60 second
    fetching_time = time.time() - start_time
    if fetching_time > (running_time * 3600):      # stop fetching 
        break                        
        
ex_time = time.time() - start_time
rows_fetched = df.shape[0] - rows_count_before
print("total call : %d times" % (call_counter))
print("fetching data time :  %s minutes %d seconds" % (int(ex_time/60), int(ex_time%60))) 
print("rows fetched : %d" % (rows_fetched))

total call : 6 times
fetching data time :  6 minutes 5 seconds
rows fetched : 95


In [4]:
df.duplicated('match_id').any()

False

### Memastikan data pertandingan yang diambil data pertandingan selesai

In [5]:
print("uncompleted match before removed : %d" % (df.isnull().sum().sum()))
df = df[df['avg_mmr'].notna()]
print("uncompleted match after removed  : %d" % (df.isnull().sum().sum()))

uncompleted match before removed : 0
uncompleted match after removed  : 0


### Mengambil dan menampilkan data hero

In [6]:
# pd.set_option('display.max_rows', None)

In [7]:
link = 'https://api.opendota.com/api/heroes'

r = requests.get(link)
data = json.loads(r.text) # load json data
    
heroes_df = pd.DataFrame(data)                    # save json data to dafaframe
heroes_df

,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",2
2,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",2
...,...,...,...,...,...,...,...
116,123,npc_dota_hero_hoodwink,Hoodwink,agi,Ranged,"[Support, Nuker, Escape, Disabler]",4
117,126,npc_dota_hero_void_spirit,Void Spirit,int,Melee,"[Carry, Escape, Nuker, Disabler]",2
118,128,npc_dota_hero_snapfire,Snapfire,str,Ranged,"[Support, Nuker, Disabler, Escape]",2
119,129,npc_dota_hero_mars,Mars,str,Melee,"[Carry, Initiator, Disabler, Durable]",2
